In [ ]:
set_idx = 1

In [ ]:
%matplotlib inline
import Ipynb_importer
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import scipy.io as sio  
import matplotlib.pyplot as plt  
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import copy as copy

from SST_loader import SST_loader
from res_disp import res_disp



    
# Settings
cuda = True
flag_window = True
window_sz = 6
flag_autoregress = False
feature_opt = 'mean'
perc_dim = []

if set_idx == 1:
    idx_train = np.array(range(0,292))
    idx_valid = np.array(range(300,320))
    idx_test = np.array(range(328,348))
elif set_idx == 2:
    idx_train = np.concatenate((np.array(range(0,272)),np.array(range(336,348))),0)
    idx_valid = np.array(range(280,300))
    idx_test = np.array(range(308,328))
elif set_idx == 3:
    idx_train = np.concatenate((np.array(range(0,252)),np.array(range(316,348))),0)
    idx_valid = np.array(range(260,280))
    idx_test = np.array(range(288,308))
elif set_idx == 4:
    idx_train = np.concatenate((np.array(range(0,232)),np.array(range(296,348))),0)
    idx_valid = np.array(range(240,260))
    idx_test = np.array(range(268,288))
elif set_idx == 5:
    idx_train = np.concatenate((np.array(range(0,212)),np.array(range(276,348))),0)
    idx_valid = np.array(range(220,240))
    idx_test = np.array(range(248,268))
elif set_idx == 6:
    idx_train = np.concatenate((np.array(range(0,192)),np.array(range(256,348))),0)
    idx_valid = np.array(range(200,220))
    idx_test = np.array(range(228,248))






if not flag_window and window_sz!=0:
    print('!!!')

n_stn = 58
flag = True
for i in range(n_stn):
    if flag:
        temp_X,temp_y,_ = SST_loader('../data/multimodel-SST_raw_'+str(i+1)+'.mat', idx_train, feature_opt, flag_window, window_sz, flag_autoregress, perc_dim)
        X_train = temp_X[idx_train,:,:,:,:]
        y_train = temp_y[idx_train,:,:]
        
        X_valid = temp_X[idx_valid,:,:,:,:]
        y_valid = temp_y[idx_valid,:,:]
        
        X_test = temp_X[idx_test,:,:,:,:]
        y_test = temp_y[idx_test,:,:]

        flag = False
    else:
        temp_X,temp_y,_ = SST_loader('../data/multimodel-SST_raw_'+str(i+1)+'.mat', idx_train, feature_opt, flag_window, window_sz, flag_autoregress, perc_dim)
        X_train = np.concatenate((X_train,temp_X[idx_train,:,:,:,:]),0)
        y_train = np.concatenate((y_train,temp_y[idx_train,:,:]),0)

        X_valid = np.concatenate((X_valid,temp_X[idx_valid,:,:,:,:]),0)
        y_valid = np.concatenate((y_valid,temp_y[idx_valid,:,:]),0)

        X_test = np.concatenate((X_test,temp_X[idx_test,:,:,:,:]),0)
        y_test = np.concatenate((y_test,temp_y[idx_test,:,:]),0)


_,_,n_lead,n_model,n_dim = X_train.shape


if flag_autoregress:
    n_model = n_model-1



In [ ]:
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).float()
X_valid = torch.from_numpy(X_valid).float()
y_valid = torch.from_numpy(y_valid).float()
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).float()

In [ ]:
class hierarchy_LSTM(nn.Module):

    def __init__(self, n_lead, flag_autoregress, n_dim, para):
        super(hierarchy_LSTM, self).__init__()
        self.n_lead = n_lead
        self.flag_autoregress = flag_autoregress
        self.n_dim = n_dim
        self.hidden_dim_bot = para.hidden_dim_bot
        self.hidden_dim_top = para.hidden_dim_top
        self.dp_rate_out = para.dp_rate_out


        self.relu = nn.ReLU()
    
        
        ## 1st level lstm
        self.lstm_bot = nn.LSTM(input_size=self.n_dim, hidden_size=self.hidden_dim_bot, num_layers=1, bidirectional=False)

        
        ## 2nd level lstm
        self.lstm_top = nn.ModuleList([])
        for l in range(self.n_lead):
            if self.flag_autoregress:
                self.lstm_top.append(nn.LSTM(input_size=(self.hidden_dim_bot)+1, hidden_size=self.hidden_dim_top, num_layers=1, bidirectional=False))
            else:
                self.lstm_top.append(nn.LSTM(input_size=(self.hidden_dim_bot), hidden_size=self.hidden_dim_top, num_layers=1, bidirectional=False))


        ## output layer
        self.fc1 = nn.ModuleList([])
        for l in range(self.n_lead):
            self.fc1.append(nn.Linear(self.hidden_dim_top, 1))
        self.out_dropout = nn.Dropout(p=self.dp_rate_out)



    def init_hidden(self,batch_size,hidden_dim,cuda): # 1 hidden, 2 cell
        if cuda:
            return (Variable(torch.zeros(1, batch_size, hidden_dim).cuda()),
                    Variable(torch.zeros(1, batch_size, hidden_dim).cuda()))
        else:
            return (Variable(torch.zeros(1, batch_size, hidden_dim)),
                    Variable(torch.zeros(1, batch_size, hidden_dim)))



    
    def forward(self, inputs_all, cuda):
        batch_size = inputs_all.size()[0]
        window_sz = inputs_all.size()[1]
          
        if self.flag_autoregress:
            inputs = inputs_all[:,:,:,0:-1,:]
        else:
            inputs = inputs_all
            
         
        x = torch.squeeze(inputs).contiguous()
        
        ## 1st level lstm
        x = x.view(-1,self.n_lead,self.n_dim)  # (n*seq_top, seq_bot, embed_dim)
        x = x.permute(1,0,2)   # (seq_bot, n*seq_top, embed_dim)
        hidden_bot = self.init_hidden(batch_size*window_sz,self.hidden_dim_bot,cuda)
        lstm_bot_out, hidden_bot = self.lstm_bot(x, hidden_bot)  # (seq_bot, n*seq_top, dim_bot)
        hidden_bot = (hidden_bot[0].detach(),hidden_bot[1].detach())
        lstm_bot_out_org = lstm_bot_out.permute(1,0,2).contiguous()   #( n*seq_top, seq_bot, dim_bot)
        lstm_bot_out_org = lstm_bot_out_org.view(batch_size, window_sz, self.n_lead, self.hidden_dim_bot)         #( n, seq_top, seq_bot, dim_bot)

        if self.flag_autoregress:
            latest_obs = inputs_all[:,:,:,-1,0].contiguous()
            latest_obs = latest_obs.view(batch_size,window_sz,self.n_lead,1)
            context = torch.cat((lstm_bot_out_org,latest_obs),3)
        else:
            context = lstm_bot_out_org
        #( n, seq_top, seq_bot, dim_bot)  
            
            
            
            
        ## 2nd level lstm
        if cuda:
            lstm_top_out = Variable(torch.zeros(window_sz,batch_size,self.n_lead,self.hidden_dim_top)).cuda()
        else:
            lstm_top_out = Variable(torch.zeros(window_sz,batch_size,self.n_lead,self.hidden_dim_top)) #( seq_top, n, seq_bot, dim_top) 
        context = context.permute(1,0,2,3)   #( seq_top, n, seq_bot, dim_bot) 
        for l in range(self.n_lead):
            hidden_top = self.init_hidden(batch_size,self.hidden_dim_top,cuda)
            lstm_top_out[:,:,l,:], hidden_top = self.lstm_top[l](torch.squeeze(context[:,:,l,:]), hidden_top)
            hidden_top = (hidden_top[0].detach(),hidden_top[1].detach())
        lstm_top_out_org = lstm_top_out.permute(1,0,2,3)   #( n, seq_top, seq_bot, dim_top) 

        ## output layer
        if cuda:
            out = Variable(torch.zeros(batch_size,window_sz,self.n_lead)).cuda()
        else:
            out = Variable(torch.zeros(batch_size,window_sz,self.n_lead))
        for l in range(self.n_lead):
            temp = self.out_dropout(lstm_top_out_org)
            temp = self.fc1[l](temp[:,:,l,:])
            out[:,:,l] = temp
        # (n, seq_top, seq_bot)
        
        return out

In [ ]:
def predict(dataloader,net,cuda):
    all_outputs = torch.zeros(0,n_lead)
    all_targets = torch.zeros(0,n_lead)
    for i, data in enumerate(dataloader):
        inputs_batch, targets_batch = data
        all_targets = torch.cat((all_targets,targets_batch[:,-1,:]),0)
        inputs_batch = Variable(torch.squeeze(inputs_batch),requires_grad=False)
        if cuda:
            inputs_batch = inputs_batch.cuda()
            all_outputs = torch.cat((all_outputs,net(inputs_batch,cuda).data.cpu()[:,-1,:]),0)
        else:
            all_outputs = torch.cat((all_outputs,net(inputs_batch,cuda).data[:,-1,:]),0)
    return all_outputs, all_targets


In [ ]:

class para_ele:
    def __init__(self,batchsize,learn_rate,hidden_dim_bot,hidden_dim_top,dp_rate_out):
        self.batchsize = batchsize
        self.learn_rate = learn_rate
        self.hidden_dim_bot = hidden_dim_bot
        self.hidden_dim_top = hidden_dim_top
        self.dp_rate_out = dp_rate_out


In [ ]:


if set_idx == 1:
    para_ele_list = [para_ele(256, 0.0010, 50, 40, 0.0000)]   # set 1 
elif set_idx == 2:
    para_ele_list = [para_ele(256, 0.0010, 10, 50, 0.0000)]   # set 2  
elif set_idx == 3:
    para_ele_list = [para_ele(128, 0.0010, 30, 10, 0.5000)]   # set 3
elif set_idx == 4:
    para_ele_list = [para_ele(128, 0.0100, 10, 10, 0.5000)]   # set 4
elif set_idx == 5:
    para_ele_list = [para_ele(128, 0.0010, 30, 10, 0.5000)]   # set 5
elif set_idx == 6:
    para_ele_list = [para_ele(256, 0.0100, 10, 10, 0.1000)]   # set 6 






In [ ]:




all_outputs_test = []
all_targets_test = []

seeds = range(15)
for para in para_ele_list:

    all_rmse_train = torch.zeros(n_lead,len(seeds))
    all_rmse_valid = torch.zeros(n_lead,len(seeds))
    all_rmse_test = torch.zeros(n_lead,len(seeds))
    
    n_trial = 0
    
    trainset = torch.utils.data.TensorDataset(X_train,y_train)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=para.batchsize, shuffle=True)
    validset = torch.utils.data.TensorDataset(X_valid,y_valid)
    validloader = torch.utils.data.DataLoader(validset, batch_size=para.batchsize, shuffle=False)
    testset = torch.utils.data.TensorDataset(X_test,y_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=para.batchsize, shuffle=False)
    
    
    
    for seed in seeds:
        n_trial += 1
    
        torch.manual_seed(seed)
        net = hierarchy_LSTM( n_lead, flag_autoregress, n_dim, para )
        criterion = nn.MSELoss()

        optimizer = torch.optim.Adam(net.parameters(), lr = para.learn_rate)
        optimizer.zero_grad()

        if cuda:
            net = net.cuda()
            criterion = criterion.cuda()

        best_loss = 1000000000000000000
        best_net = copy.deepcopy(net)
        stop_cr = 0

        for epoch in range(5000):  # loop over the dataset multiple times
            net.train()
            running_loss = 0.0
            for i, data in enumerate(trainloader):
                optimizer.zero_grad()

                inputs, targets = data
                if cuda:
                    inputs = inputs.cuda()
                    targets = targets.cuda()
                
                outputs = net(Variable(inputs),cuda)   # (n, window_sz, n_lead)
                
                loss = criterion(outputs,Variable(targets))


                loss.backward()            
                optimizer.step()
                running_loss += loss.data[0]

            if epoch%1 == 0:
                net.eval()
                
                output_valid, target_valid = predict(validloader,net,cuda)

                valid_loss = torch.mean(torch.sqrt(torch.mean((output_valid-target_valid)**2,0)))

                current_loss = valid_loss
                if current_loss < best_loss:
                    stop_cr = 0
                    best_loss = current_loss
                    best_net = copy.deepcopy(net)
                else:
                    stop_cr += 1
                if stop_cr>=10:
                    break

        net = copy.deepcopy(best_net)
        net.eval()
        



        # Testing Part
        net.eval()
               
        outputs_train, targets_train = predict(trainloader,net,cuda)
        outputs_valid, targets_valid = predict(validloader,net,cuda)
        outputs_test, targets_test = predict(testloader,net,cuda)

        all_rmse_train[:,n_trial-1] =  torch.sqrt(torch.mean((outputs_train-targets_train)**2,0))
        all_rmse_valid[:,n_trial-1] = torch.sqrt(torch.mean((outputs_valid-targets_valid)**2,0))
        all_rmse_test[:,n_trial-1] = torch.sqrt(torch.mean((outputs_test-targets_test)**2,0))
        
        
        all_outputs_test.append(outputs_test.numpy())
        all_targets_test.append(targets_test.numpy())

    
    ave_rmse_train = torch.mean(all_rmse_train,1)
    ave_rmse_valid = torch.mean(all_rmse_valid,1)
    ave_rmse_test = torch.mean(all_rmse_test,1)
    std_rmse_train = torch.std(all_rmse_train,1)
    std_rmse_valid = torch.std(all_rmse_valid,1)
    std_rmse_test = torch.std(all_rmse_test,1)

    print(para.batchsize, para.learn_rate, para.hidden_dim_bot, para.hidden_dim_top, para.dp_rate_out)
    print('%.4f, %.4f, %.4f' % (torch.mean(ave_rmse_train),torch.mean(ave_rmse_valid),torch.mean(ave_rmse_test)))
    print('-----------------------------------')
    
